In [3]:
import pandas as pd
import re
from itertools import product
from lightgbm import LGBMClassifier
import pandas as pd
import matplotlib.pylab as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from numpy import mean
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from shapely.geometry import LineString, Point
from geopy.distance import geodesic
import geopandas as gpd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [4]:

def modelCompare( X_train, y_train, X_test,  y_test):
    # Split the data into train and test sets
    # X_train, X_test, y_train, y_test = TTS(X, y, test_size=0.2, random_state=10)

    # Define the classifiers
    classifiers = [
        ("Logistic Regression", LogisticRegression()),
        ("Naive Bayes", GaussianNB()),
        ("Linear Discriminant Analysis", LinearDiscriminantAnalysis()),
        ("K-Nearest Neighbors", KNeighborsClassifier()),
        ("Decision Tree", DecisionTreeClassifier()),
        ("Random Forest", RandomForestClassifier()),
        ("Support Vector Machines", SVC(probability=True)),
        ("Gradient Boosting", GradientBoostingClassifier()),
        ("AdaBoost", AdaBoostClassifier()),
        ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss")),
        ("LightGBM", LGBMClassifier()),
        ("Quadratic Discriminant Analysis", QuadraticDiscriminantAnalysis()),
        ("Gaussian Process Classifier", GaussianProcessClassifier()),
        ("Stochastic Gradient Descent", SGDClassifier()),
        ("Linear Support Vector Machines", LinearSVC()),
        ("Multi-Layer Perceptron", MLPClassifier())
    ]

    # Train and evaluate the classifiers
    results = []
    for name, clf in classifiers:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        result = {
            "Classifier": name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "Confusion Matrix": cm
        }
        results.append(result)

    # Convert the results to a DataFrame and display
    results_df = pd.DataFrame(results)
    results_df = results_df[["Classifier", "Accuracy", "Precision", "Recall", "F1 Score","Confusion Matrix"]]
    return results_df

In [5]:
def five_fold(X, y):

    X_train,X_test,y_train,y_test = TTS(X,y,test_size=0.3,random_state=420)
    lgbm = LGBMClassifier(n_estimators=80, learning_rate=0.1, max_depth=5, num_leaves=7, min_child_samples=5,
                          reg_alpha=0.0, reg_lambda=0.0, min_split_gain=0.0, subsample=0.5, colsample_bytree=0.5,
                          subsample_freq=3, min_child_weight=0.001)

    kf = KFold(n_splits=5, shuffle=True, random_state=1)

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        lgbm.fit(X_train, y_train)
        y_pred = lgbm.predict(X_test)

        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))

    print(f"5-Fold Cross Validation Accuracy: {mean(accuracy_scores)}")
    print(f"5-Fold Cross Validation Precision: {mean(precision_scores)}")
    print(f"5-Fold Cross Validation Recall: {mean(recall_scores)}")
    print(f"5-Fold Cross Validation F1 Score: {mean(f1_scores)}")
    score = pd.DataFrame({'acc': accuracy_scores, 'pre': precision_scores, 'recall':recall_scores, 'f1': f1_scores})
    return lgbm, score

In [6]:
def getScore(y,y_pred):
    print("Accuracy:{} ".format(accuracy_score(y, y_pred)))
    print("Precision:{} ".format(precision_score(y, y_pred)))
    print("Recall:{} ".format(recall_score(y, y_pred)))
    print("F1 Score:{} ".format(f1_score(y, y_pred)))

In [7]:
def confusion(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    print("Confusion Matrix:")
    print(cm)
    return cm